In [67]:
import pandas as pd
import matplotlib.pyplot as plt
from ipywidgets import widgets

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score 
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV

# Get Data from Website

!wget https://s3.amazonaws.com/drivendata/data/57/public/train_values.csv -nc -P ./nepal/

!wget https://s3.amazonaws.com/drivendata/data/57/public/train_labels.csv -nc -P ./nepal/

!wget https://s3.amazonaws.com/drivendata/data/57/public/test_values.csv -nc -P ./nepal/


# Import Data

In [38]:
X = pd.read_csv("./nepal/train_values.csv",
                index_col='building_id',
                dtype={'geo_level_1_id': 'object',
                       'geo_level_2_id': 'object',
                       'geo_level_3_id': 'object'})

In [39]:
X.head()

,geo_level_1_id,geo_level_2_id,geo_level_3_id,count_floors_pre_eq,age,area_percentage,height_percentage,land_surface_condition,foundation_type,roof_type,...,has_secondary_use_agriculture,has_secondary_use_hotel,has_secondary_use_rental,has_secondary_use_institution,has_secondary_use_school,has_secondary_use_industry,has_secondary_use_health_post,has_secondary_use_gov_office,has_secondary_use_use_police,has_secondary_use_other
building_id,,,,,,,,,,,,,,,,,,,,,
802906,6,487,12198,2,30,6,5,t,r,n,...,0,0,0,0,0,0,0,0,0,0
28830,8,900,2812,2,10,8,7,o,r,n,...,0,0,0,0,0,0,0,0,0,0
94947,21,363,8973,2,10,5,5,t,r,n,...,0,0,0,0,0,0,0,0,0,0
590882,22,418,10694,2,10,6,5,t,r,n,...,0,0,0,0,0,0,0,0,0,0
201944,11,131,1488,3,30,8,9,t,r,n,...,0,0,0,0,0,0,0,0,0,0


In [40]:
X.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 260601 entries, 802906 to 747594
Data columns (total 38 columns):
geo_level_1_id                            260601 non-null object
geo_level_2_id                            260601 non-null object
geo_level_3_id                            260601 non-null object
count_floors_pre_eq                       260601 non-null int64
age                                       260601 non-null int64
area_percentage                           260601 non-null int64
height_percentage                         260601 non-null int64
land_surface_condition                    260601 non-null object
foundation_type                           260601 non-null object
roof_type                                 260601 non-null object
ground_floor_type                         260601 non-null object
other_floor_type                          260601 non-null object
position                                  260601 non-null object
plan_configuration                        2

In [41]:
y = pd.read_csv('nepal/train_labels.csv', index_col='building_id')['damage_grade']

# Solutions 1: Model /w one feature 

In [6]:
def housing_plot(X,y):
    def plotter(column):
        valid_rows = X[column].notna()
        plt.plot(X.loc[valid_rows, column], y[valid_rows], '.', color='k')
        plt.ylabel('Damage Level')
        plt.yticks([1,2,3])
        
    return plotter

dropdown_values = sorted(X.columns)
widgets.interact(housing_plot(X,y), column=dropdown_values);

interactive(children=(Dropdown(description='column', options=('age', 'area_percentage', 'count_families', 'cou…

In [42]:
X_height = X[['height_percentage']]
X_height.head()

,height_percentage
building_id,
802906,5
28830,7
94947,5
590882,5
201944,9


In [8]:
X_train, X_test, y_train, y_test = train_test_split(X_height, y, test_size=0.2, random_state=42)

In [9]:
one_feat_model = LogisticRegression(solver='lbfgs', multi_class='auto')
one_feat_model.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='auto',
          n_jobs=None, penalty='l2', random_state=None, solver='lbfgs',
          tol=0.0001, verbose=0, warm_start=False)

# Compare in- and out-sample metrics (f1 score)

In [10]:
y_train_pred = one_feat_model.predict(X_train)

print("In-sample f1 score:")
f1_score(y_train, y_train_pred, average='micro')

In-sample f1 score:


0.5699779355333845

In [11]:
y_test_pred = one_feat_model.predict(X_test)

print("Out-sample f1 score:")
f1_score(y_test, y_test_pred, average='micro')

Out-sample f1 score:


0.5660290477926364

# Create submission

In [23]:
def create_submission(model, X_cols=None):
    X = pd.read_csv("./nepal/test_values.csv", index_col='building_id')
    if X_cols != None:
        X = X[X_cols]
    y_pred = model.predict(X)
    submission = pd.DataFrame(y_pred, index=X.index,
                              columns=['damage_grade'])
    date_string = pd.Timestamp.utcnow().strftime(format='%Y-%m-%d_%H%M_')
    submission.to_csv(f'nepal/{date_string}submission_csv')

In [13]:
create_submission(one_feat_model, ['height_percentage'])

## Score: 0.56

# Solutions 2: All Numerical Features - Logistic Regression

In [58]:
numerical_features = list(X.describe().columns)

In [59]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [60]:
num_feat_model = LogisticRegression(solver='lbfgs', max_iter=1000, multi_class='auto')
num_feat_model.fit(X_train[numerical_features], y_train)
y_train_pred = num_feat_model.predict(X_train[numerical_features])
 
print('In-sample f1 score:')
f1_score(y_train, y_train_pred, average='micro')

In-sample f1 score:


C:\Users\konra\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)


0.5762135456638526

In [61]:
y_test_pred = num_feat_model.predict(X_test[numerical_features])

print('Out-sample f1 score:')
f1_score(y_test, y_test_pred, average='micro')

Out-sample f1 score:


0.5758523435851193

# Solutions 3: All Numerical Features - Gradient Boosting Classifier

In [79]:
from sklearn.ensemble import GradientBoostingClassifier

xg_num_model = GradientBoostingClassifier()
xg_num_model.fit(X_train[numerical_features], y_train)
y_train_pred = num_feat_model.predict(X_train[numerical_features])

print('In-sample f1 score:')
f1_score(y_train, y_train_pred, average='micro')

In-sample f1 score:


0.5762135456638526

In [80]:
y_test_pred = xg_num_model.predict(X_test[numerical_features])

print('Out-sample f1 score:')
f1_score(y_test, y_test_pred, average='micro')

Out-sample f1 score:


0.5902227509065444

In [81]:
create_submission(xg_num_model, numerical_features)

# Solutions 4: All Numerical Features - Random Forest Classifier with some GridSearchCV

In [63]:
from sklearn.ensemble import RandomForestClassifier


rand_for_model = RandomForestClassifier(n)
rand_for_model.fit(X_train[numerical_features], y_train)
y_train_pred = num_feat_model.predict(X_train[numerical_features])

print('In-sample f1 score:')
f1_score(y_train, y_train_pred, average='micro')

C:\Users\konra\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


In-sample f1 score:


0.5762135456638526

In [74]:
param_grid = {'max_depth': range(4,11,2),
              'min_samples_leaf': range(250,301,50),
              'n_estimators': range(100,251,50),
              'class_weight': ['balanced', 'balanced_subsample', None]}

gs = GridSearchCV(RandomForestClassifier(), 
                        param_grid=param_grid,
                        cv=5,
                        n_jobs=12,
                        verbose=1)



In [75]:
random_forest_model = Pipeline([
    ('gs', gs)
])

In [76]:
random_forest_model.fit(X_train[numerical_features], y_train)
y_train_pred = random_forest_model.predict(X_train[numerical_features])

print('In-sample f1 score:')
f1_score(y_train, y_train_pred, average='micro')

Fitting 5 folds for each of 96 candidates, totalling 480 fits


[Parallel(n_jobs=12)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=12)]: Done  26 tasks      | elapsed:  1.6min
[Parallel(n_jobs=12)]: Done 176 tasks      | elapsed: 13.2min
[Parallel(n_jobs=12)]: Done 426 tasks      | elapsed: 34.6min
[Parallel(n_jobs=12)]: Done 480 out of 480 | elapsed: 39.1min finished


In-sample f1 score:


0.5864543361473523

In [77]:
y_test_pred = random_forest_model.predict(X_test[numerical_features])

print('Out-sample f1 score:')
f1_score(y_test, y_test_pred, average='micro')

Out-sample f1 score:


0.583430862800023